## <font color='green'> <div align="center">In the name of God </div></font>

### <font color='red'> Author: Sayed Kamaledin Ghiasi-Shrirazi <a href="http://profsite.um.ac.ir/~k.ghiasi">(http://profsite.um.ac.ir/~k.ghiasi)</a> </font>

# Recognition accuracy experiments on the feret face recognition dataset with a regularization parameter of $10$

##### Importing general modules.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import sklearn
import scipy.stats as st
import scipy.io as sio
import time
import matplotlib.image as mpimg
import matplotlib as mpl
import os
import cv2

##### Importing modules wtitten by the author

In [2]:
from LinearDiscriminantAnalysis import LinearDiscriminantAnalysis
from LDATrainingDataPreparation import TrainingData
from NearestNeighbor import NearestNeighbor

##### Setup

In [3]:
C  = 2
K = 20
N_c = [822, 654]
N = np.sum(N_c)
maxVqIteration = 100
width = 384//2
height = 384//2
reg =  10

##### Load dataset

In [4]:
path = '../datasets/feret'
X = np.zeros ([N, height, width])
y = np.zeros (N, dtype = int)

n = 0
for png_obj in os.listdir(path + '/Male'):
    file = os.path.join(path, 'Male',png_obj)
    img = mpimg.imread (file)
    img = cv2.resize(img,(height,width))
    X[n,:,:] = np.mean(img, axis=2)
    y[n] = 0
    n += 1

for png_obj in os.listdir(path + '/Female'):
    file = os.path.join(path, 'Female',png_obj)
    img = mpimg.imread (file)
    img = cv2.resize(img,(height,width))
    X[n,:,:] = np.mean(img, axis=2)
    y[n] = 1
    n += 1

X = X.reshape ([-1, height * width]) - np.mean(X)

##### Defining a function for computing accuracy on a specific dataset (i.e. a fold)

In [5]:
def compute_accuracy(XTrain, yTrain, XTest, yTest, K, solver, metric_learning, S1, S2, regularization, min_svd):
    np.random.seed(1)
    td = TrainingData(XTrain, yTrain)

    clusAlg = sklearn.cluster.KMeans()
    clusAlg.max_iter = maxVqIteration
    td.findSubclasses(np.ones(C, dtype = int)*K, clusAlg)
    
    #for i in range(td.C):
    #    for j in range (td.Ki[i]):
    #        X = td.subclassMeans[i,j,:]
    #        img = np.zeros ([height,width, 3])
    #        img[:,:,0] = img[:,:,1] = img[:,:,2] = np.reshape(X, [height, width])
    #        plt.imshow (img)
    #        plt.show()    
    
    lda = LinearDiscriminantAnalysis(td, solver, metric_learning, S1, S2, regularization, min_svd)
    lda.fitFeatureExtractor()
    obj = lda.objective()
    
    XTrainFtr = lda.transform (XTrain)
    XTestFtr  = lda.transform (XTest)
    accNN = NearestNeighbor (XTrainFtr, yTrain, XTestFtr, yTest)
    return accNN, obj

##### Defining a function for computing average accuracy on 10 folds

In [6]:
def compute_average_accuracy (solver, metric_learning, S1, S2, reg):
    nFolds = 10
    accNN = np.zeros ([nFolds])
    accProjNN = np.zeros ([nFolds])
    obj = np.zeros ([nFolds])
    
    N, dim = X.shape
    C = max(y)+1
    Ki = np.bincount (y)
    Xc = np.zeros ([C, max(Ki), dim] )
    for c in range (C):
        Xc[c,:Ki[c],:] = X[y == c,:]
        
    np.random.seed(1)
    for fold in range (nFolds):
        idxTrn = 0
        idxTst = 0
        NTest = np.sum ((Ki+nFolds-1) // nFolds)
        NTrain = N - NTest
        XTrain = np.zeros ([NTrain, dim])
        yTrain = np.zeros ([NTrain])
        XTest = np.zeros ([NTest, dim])
        yTest = np.zeros ([NTest])
        for c in range (C):
            idx1 = fold * (Ki[c] // nFolds)
            idx2 = idx1 + (Ki[c]+nFolds-1) // nFolds
            if idx2 > Ki[c]:
                idx2 = Ki[c]
                idx1 -= 1
            XTrain1 = Xc[c,:idx1,:]
            XTrain2 = Xc[c,idx2:Ki[c],:]
            length  = Ki[c] - (idx2 - idx1)
            XTrain[idxTrn:idxTrn+length,:]  = np.concatenate((XTrain1, XTrain2), axis = 0)
            yTrain[idxTrn:idxTrn+length]    = c
            idxTrn += length
            length  = (idx2 - idx1)
            XTest[idxTst:idxTst+length,:]   = Xc[c,idx1:idx2,:]
            yTest[idxTst:idxTst+length]     = c
            idxTst += length
        (obj[fold], accNN[fold]) = \
                compute_accuracy(XTrain, yTrain, XTest, yTest, K, solver, metric_learning, S1, S2, reg, 0.00001)

    return obj, accNN

In [7]:
solver = 'svd'
S1 ='Sb'
S2 = 'St'
accNN, obj = compute_average_accuracy (solver, '', S1, S2, reg)
for (name, acc) in [('accNN', accNN)]:
    print ('')
    acc *= 100
    print (name , 'Mean: ', np.mean (acc), 'Std of Mean Accuracy: ', 
           np.std (acc), 'Objective:', np.mean(obj), 'Objective-std:', np.std(obj))


accNN Mean:  83.95973154362416 Std of Mean Accuracy:  4.446277067128021 Objective: 6.109498666053435 Objective-std: 0.07387116382758208


In [8]:
solver = 'ghiasi_svd'
S1 ='Sb'
S2 = 'St'
accNN, obj = compute_average_accuracy (solver, 'none', S1, S2, reg)
for (name, acc) in [('accNN', accNN)]:
    print ('')
    acc *= 100
    print (name , 'Mean: ', np.mean (acc), 'Std of Mean Accuracy: ', 
           np.std (acc), 'Objective:', np.mean(obj), 'Objective-std:', np.std(obj))


accNN Mean:  83.35570469798657 Std of Mean Accuracy:  3.3930804455743724 Objective: 6.109498666053435 Objective-std: 0.07387116382758879


In [9]:
solver = 'ghiasi_svd'
S1 ='Sb'
S2 = 'St'
accNN, obj = compute_average_accuracy (solver, 'eig', S1, S2, reg)
for (name, acc) in [('accNN', accNN)]:
    print ('')
    acc *= 100
    print (name , 'Mean: ', np.mean (acc), 'Std of Mean Accuracy: ', 
           np.std (acc), 'Objective:', np.mean(obj), 'Objective-std:', np.std(obj))


accNN Mean:  84.09395973154362 Std of Mean Accuracy:  4.640588969949671 Objective: 6.109498666053434 Objective-std: 0.07387116382758332


### <font color='red'> Author: Sayed Kamaledin Ghiasi-Shrirazi <a href="http://profsite.um.ac.ir/~k.ghiasi">(http://profsite.um.ac.ir/~k.ghiasi)</a> </font>